# Flowchart-Detection model
I have confirmed that it works with Python 3.11.12.

In [ ]:
# mount to g-drive
from google.colab import drive
drive.mount('/content/drive')

## 1. prepare code and dataaset
### 1.1 git clone damo-yolo model
- git clone damo-yolo model
```bash
git clone https://github.com/tinyvision/DAMO-YOLO.git
```

### 1.2  locate in G-drive
- in this notebook, I located it in /content/drive/MyDrive/programs/flow-chart-detection/

### 1.3 prepare dataset
- Prepare your own dataset in coco dataset format.  
The following directory structure is assumed in this notebook.
```
/content/drive/MyDrive/programs/flow-chart-detection
├── ckpt
│   ├── damoyolo_tinynasL20_T_436.pth
│   └── ...
├── damo-yolo
│   ├── assets
│   ├── configs
│   │   ├── damoyolo_tinynasL20_T.py
│   │   ├── ...
├── ...
├── data_coco_format_start_end_train
│   ├── annotations
│   │   └── instances_custom.json
│   └── images
│       ├── flowchart-example001.webp
│       ├── ...
├── data_coco_format_start_end_test
│   ├── annotations
│   │   └── instances_custom.json
│   └── images
│       ├── flowchart-example002.webp
│       ├── ...
```

In [ ]:
# install related libraries
!pip install cairosvg==2.8.2
!apt install tree
!pip install xmltodict==0.14.2
!pip install albumentations==2.0.6
!pip install ultralytics==8.3.137
!pip install cairosvg==2.8.2

## 2 Change the information in the config file
Follow the instructions on the following site to change the config file of the damo-yolo model.
https://github.com/tinyvision/DAMO-YOLO/blob/master/assets/CustomDatasetTutorial.md

### 2.1 Convert the custom dataset to coco format
Convert your flowchart dataset to coco data format as follows
```
{
  "categories":
  [{
      "supercategory": "text",
      "id": 1,
      "name": "text"
  },
  {
      "supercategory": "arrow",
      "id": 2,
      "name": "arrow"
  },
  ...
  ],
 "images":
  [{
      "file_name": "flowchart-example001.webp",        
      "height": 499,
      "width": 731,
      "id": 1
  },
  ...
  ],
 "annotations":
  [{
      "image_id": 1,
      "category_id": 3,
      "segmentation": [],
      "area": 5047,
      "iscrowd": 0,
      "bbox": [4,1,103,49],
      "id": 1
  },
  ...
  ]
}
```




## 2.2 Create symbolic linds

In [ ]:
# ex)
!ln -s /content/drive/MyDrive/programs/flow-chart-detection/data_coco_format_start_end_test /content/drive/MyDrive/programs/flow-chart-detection/damo-yolo/datasets/custom_coco_test
!ln -s /content/drive/MyDrive/programs/flow-chart-detection/data_coco_format_start_end_train /content/drive/MyDrive/programs/flow-chart-detection/damo-yolo/datasets/custom_coco_train

## 2.3 Add the custom dataset into DAMO-YOLO
Add the custom dataset into `damo/config/paths_catalog.py`. Note, the added dataset should contain coco in their names to declare the dataset format, e.g., here we use `custom_test_coco` and `custom_train_coco`.

```
'custom_train_coco': {
    'img_dir': 'custom_coco_train/images',
    'ann_file': 'custom_coco_train/annotations/instances_custom.json'
},
'custom_test_coco': {
    'img_dir': 'custom_coco_test/images',
    'ann_file': 'custom_coco_test/annotations/instances_custom.json'
},
```

In [ ]:
# set path to dataaset
PATH_TO_FCDetection = '/content/drive/MyDrive/programs/flow-chart-detection'
# check dataset path
!tree /content/drive/MyDrive/programs/flow-chart-detection

## 2.4 modify the config file

In this notebook, we finetune on DAMO-YOLO-Tiny like official tutorial.
- Download the DAMO-YOLO-Tiny torch model from Model Zoo
https://github.com/tinyvision/DAMO-YOLO#Model-Zoo
- Add the following pretrained model path into `damoyolo_tinynasL20_T.py`.
```
self.train.finetune_path='path/to/damoyolo_tinynasL20_T.pth'
```

Modify the custom dataset in config file. Change `coco_2017_train` and `coco_2017_test` in `damoyolo_tinynasL20_T.py` to `custom_train_coco` and `custom_test_coco` respectively around line 33.

Modify the category number in config file. Change `'num_classes': 80` `in damoyolo_tinynasL20_T.py` to `'num_classes': 9` around line64. Because in our flow-chart-detection, there is 9 categories, so we set `num_classes` to 9.

Modify the list of class names around line 77 like below.
```
self.dataset.class_names = ['text', 'arrow', 'terminator', 'data', 'process', 'decision', 'connection', 'arrow_start', 'arrow_end']
```

In [4]:
# install libraries
from PIL import Image
import cairosvg
from io import BytesIO
import cv2
from IPython.display import display
import numpy as np
import os
import glob

# 3. Model building and learning

In [ ]:
# import torch, else
import xml.etree.ElementTree as ET
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.models.detection import fcos_resnet50_fpn
from torchvision.models.detection.fcos import FCOSHead
import cv2
import numpy as np
from PIL import Image
import xmltodict
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os

In [ ]:
# install libraries for damo-yolo
%cd /content/drive/MyDrive/programs/flow-chart-detection/damo-yolo
!pip install -r requirements.txt
import os
os.environ['PYTHONPATH'] = f"{os.getcwd()}:{os.environ.get('PYTHONPATH', '')}"

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo')
print("sys.path ", sys.path)
!export PYTHONPATH="$PYTHONPATH:/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo"
!echo $PYTHONPATH
!ls


In [ ]:
!nvidia-smi

In [ ]:
# do fine-tuning. for detail, see damo-yolo official site.
# ex)
!torchrun --nproc_per_node=1 tools/train.py -f configs/damoyolo_tinynasL20_T.py --local_rank 0 --tea_ckpt /content/drive/MyDrive/programs/flow-chart-detection/ckpt/damoyolo_tinynasL20_T_436.pth

## 4. Visualize training results

In [ ]:
# visualize results
import re
import matplotlib.pyplot as plt

# read files. change this PATH to yours.
file_path = '/content/drive/MyDrive/programs/flow-chart-detection/damo-yolo/workdirs/damoyolo_tinynasL20_T/2025-04-23-1819'

with open(file_path, 'r') as file:
    lines = file.readlines()

# get AP, AR
epochs = []
ap_values = []
ar_values = []
current_epoch = None  # initialize current_epoch

for line in lines:
    # get epoch info
    epoch_match = re.search(r'epoch: (\d+)/', line)
    if epoch_match:
        current_epoch = int(epoch_match.group(1))

    # confirm current_epoch is defined or not
    if current_epoch is not None:
        # get AP
        ap_match = re.search(r'Average Precision\s+\(AP\)\s+@\[\s*IoU=0\.50:0\.95.*=\s+([\d\.]+)', line)
        if ap_match:
            ap_values.append(float(ap_match.group(1)))
            epochs.append(current_epoch)

        # get AR
        ar_match = re.search(r'Average Recall\s+\(AR\)\s+@\[\s*IoU=0\.50:0\.95.*=\s+([\d\.]+)', line)
        if ar_match:
            ar_values.append(float(ar_match.group(1)))

# debug
print(f"Number of epochs: {len(epochs)}")
print(f"Number of AP values: {len(ap_values)}")
print(f"Number of AR values: {len(ar_values)}")

# Adjustment for inconsistent data counts
min_length = min(len(epochs), len(ap_values), len(ar_values))
epochs = epochs[:min_length]
ap_values = ap_values[:min_length]
ar_values = ar_values[:min_length]

# make graph
plt.figure(figsize=(10, 6))
plt.plot(epochs, ap_values, label='Average Precision (AP)', marker='o')
plt.plot(epochs, ar_values, label='Average Recall (AR)', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('AP and AR over Epochs')
plt.legend()
plt.grid(True)
plt.show()

## 5. test and get json file

In [ ]:
# test for your dataset
!torchrun --nproc_per_node=1 tools/eval.py \
  -f configs/damoyolo_tinynasL20_T_eval.py \
  --ckpt /content/drive/MyDrive/programs/flow-chart-detection/damo-yolo/workdirs/damoyolo_tinynasL20_T/epoch_3000_ckpt.pth \
  --fuse

In [14]:
# convert json data including all images to each image json file
import json
from collections import defaultdict
import os

# Input file and output directory
input_file = os.path.join(PATH_TO_FCDetection, 'damo-yolo/results/custom_eval/inference/custom_test_coco/bbox.json')
output_dir = os.path.join(PATH_TO_FCDetection, 'damo-yolo/results/each_images/')
os.makedirs(output_dir, exist_ok=True)

# Load bbox-2.json
with open(input_file, 'r') as f:
    data = json.load(f)

# Group annotations by image_id
grouped = defaultdict(list)
for i, ann in enumerate(data):
    grouped[ann['image_id']].append({
        "id": i + 1,
        "image_id": ann['image_id'],
        "category_id": ann['category_id'],
        "bbox": [round(x, 2) for x in ann['bbox']],
        "score": round(ann['score'], 3),
        "area": round(ann['bbox'][2] * ann['bbox'][3], 2),
        "segmentation": [],
        "iscrowd": 0
    })

# Dummy image info (replace width/height/file_name with actual values if available)
# Here we assume all images are 1920x1080 and named as "image_<id>.png"
for image_id, annotations in grouped.items():
    image_info = {
        "id": image_id,
        "file_name": f"image_{image_id}.png",
        "width": 1920,
        "height": 1080
    }
    output_data = {
        "images": [image_info],
        "annotations": annotations
    }

    # Save as individual JSON file per image
    output_path = os.path.join(output_dir, f"image_{image_id}.json")
    with open(output_path, 'w') as out_f:
        json.dump(output_data, out_f, indent=2)
